# सत्र 6 – बहु-चरण पाइपलाइन (योजना → निष्पादन → परिष्कृत)

रूटिंग फ़ंक्शंस का उपयोग करके प्रत्येक चरण के लिए मॉडल का चयन करता है, फिर अंतिम उत्तर को परिष्कृत करता है।


### व्याख्या: निर्भरता स्थापना
`foundry-local-sdk` और `openai` को स्थापित करता है जो प्रति-चरण चैट कॉल्स के लिए आवश्यक हैं। इसे दोबारा चलाना सुरक्षित है।


# परिदृश्य
बहु-चरणीय मॉडल पाइपलाइन जो: (1) किसी कार्य को अलग-अलग चरणों में योजना बनाती है, (2) प्रत्येक चरण को इरादे-आधारित मॉडल चयन के साथ निष्पादित करती है, (3) अंतिम संश्लेषित उत्तर को परिष्कृत करती है। दक्षता बनाए रखते हुए गुणवत्ता को संरक्षित करने के लिए विषम SLM क्षमताओं को जोड़ने का प्रदर्शन करती है।


In [13]:
!pip install -q foundry-local-sdk openai

### व्याख्या: मुख्य आयात
इंटेंट डिटेक्शन के लिए regex आयात करता है, प्रति-अलियास अटैचमेंट के लिए Foundry Local मैनेजर, और चैट पूर्णताओं के लिए OpenAI क्लाइंट आयात करता है।


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: क्षमता कैटलॉग और नियम  
एक क्षमता-संवेदनशील कैटलॉग और regex नियमों को परिभाषित करता है, जो चरण पाठ को इरादे श्रेणियों (कोड, सारांश, वर्गीकरण, सामान्य) से जोड़ने के लिए उपयोग किए जाते हैं। जब कई मॉडल किसी इरादे का समर्थन करते हैं, तो कम प्राथमिकता वाले मान टाई में जीतते हैं।


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### व्याख्या: इरादा, मॉडल चयन और चैट हेल्पर
प्रदान करता है:
- `detect_intent` रेगुलर एक्सप्रेशन आधारित वर्गीकरण के लिए।
- `pick_model` क्षमता + प्राथमिकता के आधार पर सबसे अच्छा उपनाम चुनने के लिए।
- `chat` सुविधा व्रैपर जो प्रत्येक उपनाम के लिए एक क्लाइंट बनाता है और एकल-टर्न प्रतिक्रिया लौटाता है।


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### व्याख्या: मल्टी-स्टेप पाइपलाइन फ़ंक्शन
पाइपलाइन को लागू करता है: योजना बनाना → चरणों को पार्स करना → प्रत्येक को इरादे-आधारित रूटिंग के साथ निष्पादित करना → संयुक्त आउटपुट को परिष्कृत करना। निरीक्षण या मूल्यांकन के लिए संरचित डिक्शनरी लौटाता है।


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### व्याख्या: उदाहरण कार्य चलाएँ  
यह एक पुनर्गठन-उन्मुख कार्य पर पूरी पाइपलाइन को चलाता है, जो मिश्रित इरादों (कोड + सारांश) को दर्शाता है। परिणाम डिक्शनरी में कच्ची योजना, प्रत्येक चरण के आउटपुट चुने गए मॉडल उपनामों के साथ, और अंतिम संश्लेषित उत्तर दिखाया गया है।


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### व्याख्या: परिणाम ऑब्जेक्ट प्रदर्शित करें  
संरचित पाइपलाइन आउटपुट को त्वरित निरीक्षण या आगे के मूल्यांकन (जैसे, चरण की गुणवत्ता या परिष्करण प्रभावशीलता को मापने) के लिए दिखाता है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
